In [1]:
## This is demo for kuka reaching a desired point with diff_qp
## Author : Avadesh Meduri
## Date : 25/02/2022
import time
import numpy as np
import pinocchio as pin
from robot_properties_kuka.config import IiwaConfig

import meshcat
import meshcat.transformations as tf
import meshcat.geometry as g

from diff_pin_costs import DiffFrameTranslationCost, DiffFrameVelocityCost
from inverse_qp import IOC

import torch
from torch.autograd import Function
from torch.nn import functional as F

In [2]:
robot = IiwaConfig.buildRobotWrapper()
model, data = robot.model, robot.data
f_id = model.getFrameId("EE")

In [3]:
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=False)
viz.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [4]:
dtc = DiffFrameTranslationCost.apply
dvc = DiffFrameVelocityCost.apply

def quadratic_loss(q_pred, x_des, nq, n_col):
    loss = 3.5e1*torch.linalg.norm(dtc(q_pred[-2*nq:], model, data, f_id) - x_des)
    loss += 1e0*torch.linalg.norm(dvc(q_pred[-2*nq:], torch.zeros(nq), model, data, f_id)) # asking for zero velocity
    loss += 1e-2*torch.linalg.norm(q_pred[-2*nq:-nq]) # joint regularization
    
    for i in range(n_col):    
        loss += 1e0*torch.linalg.norm(dtc(q_pred[(3*i)*nq: (3*i+2)*nq], model, data, f_id) - x_des)
        loss += 5e-1*torch.linalg.norm(dvc(q_pred[(3*i)*nq: (3*i+2)*nq], q_pred[(3*i+2)*nq:(3*i+3)*nq], model, data, f_id)) # asking for zero velocity
        loss += 1e-2*torch.linalg.norm(q_pred[(3*i+2)*nq: (3*i+3)*nq]) # control regularization
        loss += 2e-1*torch.linalg.norm(q_pred[(3*i+1)*nq: (3*i+2)*nq]) # velocity regularization
        loss += 3e-3*torch.linalg.norm(q_pred[(3*i)*nq: (3*i+1)*nq]) # joint regularization
        
        if i < n_col - 1:
            loss += 5e-2*torch.linalg.norm(torch.subtract(q_pred[(3*i+2)*nq: (3*i+3)*nq], \
                                                          q_pred[(3*i+5)*nq: (3*i+6)*nq]))

    return loss

In [16]:
nq = model.nq
nv = model.nv
q0 = [np.pi/16.0, -np.pi/16.0, 0, 0, 0, 0, 0]
x_init = np.concatenate([q0, pin.utils.zero(model.nv)])

n_col = 5
u_max = [2.5,2.5,2.5, 1.5, 1.5, 1.5, 1.0]
dt = 0.05

isvec = True
lr = 1e-1
max_eps = 100

In [17]:
buffer_size = 700

x_train_init = torch.zeros((buffer_size,len(x_init)))
x_train_des = torch.zeros((buffer_size, 3))

n_vars = 3*nq*n_col + 2*nq
if not isvec:
    y_train = torch.zeros((buffer_size, n_vars**2 + n_vars))
else:
    y_train = torch.zeros((buffer_size, 2*n_vars))

# q_des_arr = np.array([[ 0.3009,  1.1532,  1.7729,  1.7383,  1.2195, -0.0204,  0.0593],
#                       [1.3737, -0.9711, 1.6139, 1.2188, 1.5669, 0.1236, 0.2565]])
q_des_arr = np.array([[ 0.7,  3.3103582e-01,  1.2883641e-03, -8.7180752e-01,-2.8762153e-05,
                       -1.1160310e-03, 8.9641966e-05]])
x_des_arr = torch.tensor([[0.5, -0.5, 0.4], [0.5, 0.4, 0.6], [0.2, -0.7, 0.4], [0.7, 0.4, 0.5]])



In [12]:
for k in range(buffer_size):
    ioc = IOC(n_col, nq, u_max, dt, eps = 1.0, isvec=isvec)
    optimizer = torch.optim.Adam(ioc.parameters(), lr=lr)

    if k % 15 == 0:
        
        x_des = x_des_arr[np.random.randint(len(x_des_arr))] + 0.2*2*(np.random.rand(3) - [0.2, 0.3, 0.2])
    
        viz.viewer["box"].set_object(g.Sphere(0.05), 
                         g.MeshLambertMaterial(
                             color=0xff22dd,
                             reflectivity=0.8))
        viz.viewer["box"].set_transform(tf.translation_matrix(x_des.detach().numpy()))

        x_init = np.zeros(2*nq)
        x_init[0:nq] = q_des_arr[np.random.randint(len(q_des_arr))]
        x_init[0:nq] -= np.hstack((0.4*2*(np.random.rand(3) - 0.5), 0.3*2*(np.random.rand(4) - 0.5)))
        x_init[0] = 1.5*2*(np.random.rand() - 0.5) 
        x_init[nq:] = 0.4*2*(np.random.rand(nv) - 0.5)
    
    else:
        x_init = x_pred[-2*nq:]

    x_train_init[k] = torch.tensor(x_init)
    x_train_des[k] = x_des
    
    i = 0
    loss = 1000.
    old_loss = 10000.
    
    while loss > 0.03 and i < max_eps and abs(old_loss - loss) > 5e-4:
        x_pred = ioc(x_init) 
        old_loss = loss
        loss = quadratic_loss(x_pred, x_des, nq, n_col)
        print("Index :" + str(k) + "/" + str(buffer_size) + " Iteration :" + str(i) + "/" + str(max_eps) +  " loss is : " + str(loss.detach().numpy()), end = '\r', flush = True)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1
    
    x_pred = ioc(x_init).detach().numpy()
    for i in range(n_col+1):
        q = x_pred[3*nq*i:3*nq*i + nq]
        dq = x_pred[3*nq*i + nq:3*nq*i + 2*nq]

        pin.forwardKinematics(model, data, q, dq, np.zeros(nv))
        pin.updateFramePlacements(model, data)

        viz.display(q)
        time.sleep(0.01)
    
    # storing the weights and x_nom
    y_train[k] = torch.hstack((ioc.weight.flatten(), ioc.x_nom))
    

In [5]:
x_train = torch.hstack((x_train_init, x_train_des)).float()
y_train = y_train.detach().float()

torch.save(x_train, "./data/x_train1.pt")
torch.save(y_train, "./data/y_train1.pt")

NameError: name 'x_train_init' is not defined

In [7]:
x_train = torch.load("./data/x_train1.pt")
y_train = torch.load("./data/y_train1.pt")

# x_train_loaded2 = torch.load("./data/x_train2.pt")
# y_train_loaded2 = torch.load("./data/y_train2.pt")

# x_train = torch.vstack((x_train_loaded, x_train))
# y_train = torch.vstack((y_train_loaded, y_train))

# x_train = torch.vstack((x_train_loaded2, x_train))
# y_train = torch.vstack((y_train_loaded2, y_train))

print(x_train.shape)

torch.Size([700, 17])


In [8]:
m = torch.mean(y_train, 0)
std = torch.std(y_train, 0)
z = torch.where(std == 0) ## checking where std and mean are zero
std[z] = 1.0
y_train_norm = (y_train - m)/std

In [17]:
# print(m, std)
torch.save(m, "./data/mean.pt")
torch.save(std, "./data/std.pt")

In [9]:
class Net(torch.nn.Module):

    def __init__(self, inp_size, out_size):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(inp_size, 512)
        self.fc2 = torch.nn.Linear(512, 512)
        self.out = torch.nn.Linear(512, out_size)

    def forward(self, x):
       
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = self.out(x)
        return x

In [11]:
nn = Net(x_train.shape[1], y_train.shape[1])
# nn.load_state_dict(torch.load("./models/test1"))
# nn = nn.cuda()
# x_train = x_train.cuda()
# y_train_norm = y_train_norm.cuda()
# y_train = y_train.cuda()
print(x_train.shape)

lr = 1e-5
eps = int(6e4)
batch_size = 256
optimizer = torch.optim.Adam(nn.parameters(), lr=lr)
loss = torch.nn.L1Loss() #torch.nn.MSELoss() #torch.nn.HuberLoss()

for i in range(eps):
    
    ind = np.random.randint(0,len(x_train), size = batch_size)
    x_train_batch = x_train[ind]
    y_train_batch = y_train_norm[ind]
#     y_train_batch = y_train[ind]

    y_pred = nn(x_train_batch)
    error = loss(y_pred, y_train_batch) 
    print("The iteration number : " + str(i) + " The loss is :" + str(error.cpu().detach().numpy()), end='\r', flush  = True)
    optimizer.zero_grad()
    error.backward()
    optimizer.step()

torch.Size([700, 17])


In [12]:
nn = nn.cpu()
x_train = x_train.cpu()
y_train_norm = y_train_norm.cpu()

In [13]:
k = np.random.randint(buffer_size)
print(k)
x_des = x_train[k]
viz.viewer["box"].set_object(g.Sphere(0.05), 
                         g.MeshLambertMaterial(
                             color=0xff22dd,
                             reflectivity=0.8))
viz.viewer["box"].set_transform(tf.translation_matrix(x_des[-3:].detach().numpy()))

# x_des = x_train[0]
print(x_des[-3:])

pred_norm = nn(x_des)
pred = pred_norm * std + m
# pred = y_train[k]

if not isvec:
    ioc.weight = torch.nn.Parameter(torch.reshape(pred[0:n_vars**2], (n_vars, n_vars)))
    ioc.x_nom = torch.nn.Parameter(pred[n_vars**2:])
else:
    ioc.weight = torch.nn.Parameter(pred[0:n_vars])
    ioc.x_nom = torch.nn.Parameter(pred[n_vars:])

x_pred = ioc((x_des[:-3]).detach().numpy()) 
x_pred = x_pred.detach().numpy()

plt_des = np.zeros((n_col+1, 3))

for i in range(n_col+1):
    q = x_pred[3*nq*i:3*nq*i + nq]
    dq = x_pred[3*nq*i + nq:3*nq*i + 2*nq]

    pin.forwardKinematics(model, data, q, dq, np.zeros(nv))
    pin.updateFramePlacements(model, data)
    
    plt_des[i] = data.oMf[f_id].translation

    viz.display(q)
    time.sleep(0.05)

NameError: name 'buffer_size' is not defined

In [20]:
k = np.random.randint(buffer_size)
print(k)
x_in = x_train[k]
x_des = x_in[-3:]
print(x_des)

viz.viewer["box"].set_object(g.Sphere(0.05), 
                         g.MeshLambertMaterial(
                             color=0xff22dd,
                             reflectivity=0.8))
viz.viewer["box"].set_transform(tf.translation_matrix(x_des[-3:].detach().numpy()))


for j in range(20):
    ioc = IOC(n_col, nq, u_max, dt, eps = 1.0, isvec=isvec)
    optimizer = torch.optim.Adam(ioc.parameters(), lr=lr)

    
    pred_norm = nn(x_in)
    pred = pred_norm * std + m
    
    if not isvec:
        ioc.weight = torch.nn.Parameter(torch.reshape(pred[0:n_vars**2], (n_vars, n_vars)))
        ioc.x_nom = torch.nn.Parameter(pred[n_vars**2:])
    else:
        ioc.weight = torch.nn.Parameter(pred[0:n_vars])
        ioc.x_nom = torch.nn.Parameter(pred[n_vars:])

#         print(torch.relu(ioc.weight)*ioc.x_nom)
#         print("--------------------------------------------------")
    x_pred = ioc((x_in[:-3]).detach().numpy()) 
    x_pred = x_pred.detach().numpy()


    for i in range(n_col+1):
        q = x_pred[3*nq*i:3*nq*i + nq]
        dq = x_pred[3*nq*i + nq:3*nq*i + 2*nq]

        pin.forwardKinematics(model, data, q, dq, np.zeros(nv))
        pin.updateFramePlacements(model, data)

        viz.display(q)
        time.sleep(0.05)
    
    x_in[0:2*nq] = torch.tensor(x_pred[-2*nq:])
#     print(x_pred[-nq:])


217
tensor([ 0.7499, -0.2994,  0.5863])


In [19]:
torch.save(nn.state_dict(), "./models/test1")
print("saved ...")

saved ...


In [ ]:
from matplotlib import pyplot as plt

plt.plot(x_pred[2*nq + 1:: 3*nq], label = "joint torque")
plt.legend()
plt.grid()

In [ ]:
plt_des = np.array(plt_des)
plt.plot(plt_des[:,0], label = "x_pos")
plt.grid()
plt.legend()

In [ ]:
plt_des = np.array(plt_des)
plt.plot(plt_des[:,1], label = "y_pos")
plt.grid()
plt.legend()

In [ ]:
plt_des = np.array(plt_des)
plt.plot(plt_des[:,2], label = "z_pos")
plt.grid()
plt.legend()

In [26]:
from mim_data_utils import DataLogger, DataReader
reader = DataReader('test.mds')

for i in range(len(reader.data['joint_positions'])):
    q = reader.data['joint_positions'][i]
    viz.display(q)
#     time.sleep(0.05)